# 0. Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install openai

import pandas as pd
import csv
import os
from openai import OpenAI
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
import numpy as np

In [3]:
# 配置API密钥
api_key = "sk-hBeAQRKWMQ2SDP89FKl07H0TrJpR9eAzv5JOiDRzbbT3BlbkFJLLIvw8y4m2N2i3F_H3vSpa3rBk58Yj9UdUmflWT5gA"
# 配置模型参数
gpt_model = "gpt-4-1106-preview"
temperature = 0.1
top_p = 0.5
penal = 0.0
max_input_token_length = 4096
# 初始化OpenAI客户端
client = OpenAI(api_key=api_key)

In [4]:
# 加载数据集
test = pd.read_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/test.csv')
train = pd.read_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/train.csv')

# 1.1 Functions for Zeroshot Prompting and Prediction

In [5]:
# 构造函数：Zeroshot Prompting
def zeroshot_prompting(dialogue):
    system_prompt = """
    I will provide you with a dialogue. \
    Please determine if it contains elements of mental manipulation. \
    Just answer with 'Yes' or 'No', and don't add anything else.\n
    """
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=penal,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": dialogue,
            }
        ]
    )
    res = response.choices[0].message.content
    if 'yes' in res.lower():
        return 1
    elif 'no' in res.lower():
        return 0

In [6]:
# 构造函数：Zeroshot Prediction
def zeroshot_prediction(test_data):
    targets = [int(v) for v in test_data['Manipulative'].values]
    preds = []
    for idx, row in test_data.iterrows():
        dialogue = row['Dialogue']
        pred = zeroshot_prompting(dialogue)
        preds.append(pred)
        test_data.at[idx, 'Prediction'] = pred
    test_data.to_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/zeroshot_prediction.csv', index=False)
    # 评价指标
    accuracy = accuracy_score(targets, preds)
    precision = precision_score(targets, preds, zero_division=0)
    recall = recall_score(targets, preds, zero_division=0)
    weighted_f1 = f1_score(targets, preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(targets, preds, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(targets, preds)
    FP = conf_matrix[0][1]  # False Positives: predicted 1, actual 0
    FN = conf_matrix[1][0]  # False Negatives: predicted 0, actual 1
    # 输出结果
    print(f"- Accuracy = {accuracy:.3f}")
    print(f"- Precision = {precision:.3f}")
    print(f"- Recall = {recall:.3f}")
    print(f"- Weighted F1-Score = {weighted_f1:.3f}")
    print(f"- Macro F1-Score = {macro_f1:.3f}")
    print(f"- Confusion Matrix = \n{conf_matrix}")
    print(f"- False Positives (FP) = {FP}")
    print(f"- False Negatives (FN) = {FN}")

# 1.2 Functions for Fewshot Prompting and Prediction

In [7]:
# 构造函数：Fewshot Prompting
def fewshot_prompting(manip_examples, nonmanip_examples, dialogue):
    example_list = []
    total_example_num = len(manip_examples) + len(nonmanip_examples)
    count_example = 0
    # 处理包含心理操控的示例
    for idx, row in manip_examples.iterrows():
        count_example += 1
        example = [
            {"role": "user", "content": f"Example {count_example}:\n{row['Dialogue']}"},
            {"role": "assistant", "content": "Yes"},
        ]
        example_list.extend(example)
    # 处理不包含心理操控的示例
    for idx, row in nonmanip_examples.iterrows():
        count_example += 1
        example = [
            {"role": "user", "content": f"Example {count_example}:\n{row['Dialogue']}"},
            {"role": "assistant", "content": "No"},
        ]
        example_list.extend(example)
    # 构建系统提示和消息列表
    system_prompt = f"""I will provide you with a dialogue. Please determine if it contains
    elements of mental manipulation. Just answer with 'Yes' or 'No', and don't add anything else.
    Here are {total_example_num} examples:\n"""
    messages = [{"role": "system",
                 "content": system_prompt}]
    messages += example_list
    messages.append({"role": "user",
                     "content": dialogue})
    # 调用OpenAI API生成响应
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=penal,
        messages=messages
    )
    # 处理API响应
    res = response.choices[0].message.content
    # 判断结果
    if 'yes' in res.lower():
        return 1
    elif 'no' in res.lower():
        return 0

In [8]:
# 构造函数：Fewshot Prediction
def fewshot_prediction(test_data, manip_examples, nonmanip_examples):
    targets = [int(v) for v in test_data['Manipulative'].values]
    preds = []
    for idx, row in test_data.iterrows():
        dialogue = row['Dialogue']
        pred = fewshot_prompting(manip_examples, nonmanip_examples, dialogue)
        preds.append(pred)
        test_data.at[idx, 'Prediction'] = pred
    test_data.to_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/fewshot_prediction.csv', index=False)
    # 评价指标
    accuracy = accuracy_score(targets, preds)
    precision = precision_score(targets, preds, zero_division=0)
    recall = recall_score(targets, preds, zero_division=0)
    weighted_f1 = f1_score(targets, preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(targets, preds, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(targets, preds)
    FP = conf_matrix[0][1]  # False Positives: predicted 1, actual 0
    FN = conf_matrix[1][0]  # False Negatives: predicted 0, actual 1
    # 输出结果
    print(f"- Accuracy = {accuracy:.3f}")
    print(f"- Precision = {precision:.3f}")
    print(f"- Recall = {recall:.3f}")
    print(f"- Weighted F1-Score = {weighted_f1:.3f}")
    print(f"- Macro F1-Score = {macro_f1:.3f}")
    print(f"- Confusion Matrix = \n{conf_matrix}")
    print(f"- False Positives (FP) = {FP}")
    print(f"- False Negatives (FN) = {FN}")

In [9]:
# 构造函数：Example Selection
def select_examples(data, k_manip, k_nonmanip):
    manip_list = data[data['Manipulative'] == 1].index.tolist()
    # randomly select k_manip examples from manip_list
    selected_manip_idx = np.random.choice(manip_list, k_manip, replace=False)
    nonmanip_list = data[data['Manipulative'] == 0].index.tolist()
    # randomly select k_nonmanip examples from nonmanip_list
    selected_nonmanip_idx = np.random.choice(nonmanip_list, k_nonmanip, replace=False)
    manip_examples = data.loc[selected_manip_idx]
    nonmanip_examples = data.loc[selected_nonmanip_idx]
    # Combine and export to CSV
    fewshot_examples = pd.concat([manip_examples, nonmanip_examples])
    fewshot_examples.to_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/fewshot_examples.csv', index=False)
    return manip_examples, nonmanip_examples

# 1.3 Functions for CoT Prompting and Prediction

In [10]:
# 构造函数：CoT Prompting
def cot_prompting(dialogue):
    system_prompt = """
    I will provide you with a dialogue. \
    Please determine if it contains elements of mental manipulation. \
    Just answer with 'Yes' or 'No', and don't add anything else. \
    Let's think step by step. \n
    """
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=penal,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": dialogue,
            }
        ]
    )
    res = response.choices[0].message.content
    if 'yes' in res.lower():
        return 1
    elif 'no' in res.lower():
        return 0

In [11]:
# 构造函数：CoT Prediction
def cot_prediction(test_data):
    targets = [int(v) for v in test_data['Manipulative'].values]
    preds = []
    for idx, row in test_data.iterrows():
        dialogue = row['Dialogue']
        pred = cot_prompting(dialogue)
        preds.append(pred)
        test_data.at[idx, 'Prediction'] = pred
    test_data.to_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/cot_prediction.csv', index=False)
    # 评价指标
    accuracy = accuracy_score(targets, preds)
    precision = precision_score(targets, preds, zero_division=0)
    recall = recall_score(targets, preds, zero_division=0)
    weighted_f1 = f1_score(targets, preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(targets, preds, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(targets, preds)
    FP = conf_matrix[0][1]  # False Positives: predicted 1, actual 0
    FN = conf_matrix[1][0]  # False Negatives: predicted 0, actual 1
    # 输出结果
    print(f"- Accuracy = {accuracy:.3f}")
    print(f"- Precision = {precision:.3f}")
    print(f"- Recall = {recall:.3f}")
    print(f"- Weighted F1-Score = {weighted_f1:.3f}")
    print(f"- Macro F1-Score = {macro_f1:.3f}")
    print(f"- Confusion Matrix = \n{conf_matrix}")
    print(f"- False Positives (FP) = {FP}")
    print(f"- False Negatives (FN) = {FN}")

# 1.4 Functions for Two-intent Prompting and Prediction

In [12]:
# 构造函数：Person1意图检测
def intent_p1(data):
    system_prompt = """
    I will provide you with a dialogue. \
    Please summarize the intent \
    of the statement made by Person1 \
    in one sentence. \n
    """
    def analyze_dialogue(dialogue):
        response = client.chat.completions.create(
            model=gpt_model,
            temperature=temperature,
            top_p=top_p,
            frequency_penalty=penal,
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": dialogue,
                }
            ]
        )
        return response.choices[0].message.content
    data['Intent_p1'] = data['Dialogue'].apply(analyze_dialogue)
    return data

In [13]:
# 构造函数：Person2意图检测
def intent_p2(data):
    system_prompt = """
    I will provide you with a dialogue. \
    Please summarize the intent \
    of the statement made by Person2 \
    in one sentence. \n
    """
    def analyze_dialogue(dialogue):
        response = client.chat.completions.create(
            model=gpt_model,
            temperature=temperature,
            top_p=top_p,
            frequency_penalty=penal,
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": dialogue,
                }
            ]
        )
        return response.choices[0].message.content
    data['Intent_p2'] = data['Dialogue'].apply(analyze_dialogue)
    return data

In [14]:
# 构造函数：Two-intent Prompting
def twointent_prompting(dialogue, intent_p1, intent_p2):
    # system prompt
    system_prompt = """
    I will provide you with a dialogue \
    and intent of person1, \
    and intent of person2. \
    Please carefully analyze the dialogue and intents, \
    and determine if it contains elements of mental manipulation. \
    Just answer with 'Yes' or 'No', \
    and don't add anything else. \n
    """
    # 构造user prompt
    user_input = f"{dialogue} {intent_p1} {intent_p2}"
    response = client.chat.completions.create(
        model=gpt_model,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=penal,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_input
            }
        ]
    )
    res = response.choices[0].message.content
    if 'yes' in res.lower():
        return 1
    elif 'no' in res.lower():
        return 0

In [15]:
# 构造函数：Two-intent Prediction
def twointent_prediction(test_data):
    targets = [int(v) for v in test_data['Manipulative'].values]
    preds = []
    for idx, row in test_data.iterrows():
        intent_p1 = row['Intent_p1']
        intent_p2 = row['Intent_p2']
        dialogue = row['Dialogue']
        pred = twointent_prompting(dialogue, intent_p1, intent_p2)
        preds.append(pred)
        test_data.at[idx, 'Prediction'] = pred
    test_data.to_csv('/content/drive/MyDrive/COLING2025 MetanlManip Intent/dataset/twointent_prediction.csv', index=False)
    # 评价指标
    accuracy = accuracy_score(targets, preds)
    precision = precision_score(targets, preds, zero_division=0)
    recall = recall_score(targets, preds, zero_division=0)
    weighted_f1 = f1_score(targets, preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(targets, preds, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(targets, preds)
    FP = conf_matrix[0][1]  # False Positives: predicted 1, actual 0
    FN = conf_matrix[1][0]  # False Negatives: predicted 0, actual 1
    # 输出结果
    print(f"- Accuracy = {accuracy:.3f}")
    print(f"- Precision = {precision:.3f}")
    print(f"- Recall = {recall:.3f}")
    print(f"- Weighted F1-Score = {weighted_f1:.3f}")
    print(f"- Macro F1-Score = {macro_f1:.3f}")
    print(f"- Confusion Matrix = \n{conf_matrix}")
    print(f"- False Positives (FP) = {FP}")
    print(f"- False Negatives (FN) = {FN}")

# 2.1 Zeroshot Prompting and Prediction

In [16]:
# Zeroshot Prompting预测
print("------Baseline 1: Classification Result of Zeroshot Prompting------")
zeroshot_prediction(test)

------Baseline 1: Classification Result of Zeroshot Prompting------
- Accuracy = 0.677
- Precision = 0.813
- Recall = 0.691
- Weighted F1-Score = 0.687
- Macro F1-Score = 0.649
- Confusion Matrix = 
[[174  96]
 [187 418]]
- False Positives (FP) = 96
- False Negatives (FN) = 187


# 2.2 Fewshot Prompting and Prediction

In [17]:
# 选择example
manip_examples, nonmanip_examples = select_examples(train, k_manip=1, k_nonmanip=2)
print("------Fewshot Selected Examples------")
print(manip_examples)
print(nonmanip_examples)

------Fewshot Selected Examples------
                                              Dialogue  Manipulative
838  Person1: Goddamn you, girl! You've made me mak...             1
                                               Dialogue  Manipulative
684   Person1: A million things could have caused th...             0
1177  Person1: You need to call your dad.\nPerson2: ...             0


In [18]:
# Fewshot Prompting预测
print("------Baseline 2: Classification Result of Fewshot Prompting------")
fewshot_prediction(test, manip_examples, nonmanip_examples)

------Baseline 2: Classification Result of Fewshot Prompting------
- Accuracy = 0.687
- Precision = 0.819
- Recall = 0.702
- Weighted F1-Score = 0.696
- Macro F1-Score = 0.659
- Confusion Matrix = 
[[176  94]
 [180 425]]
- False Positives (FP) = 94
- False Negatives (FN) = 180


# 2.3 CoT Prompting and Prediction

In [19]:
# CoT Prompting预测
print("------Baseline 3: Classification Result of CoT Prompting------")
cot_prediction(test)

------Baseline 3: Classification Result of CoT Prompting------
- Accuracy = 0.703
- Precision = 0.815
- Recall = 0.737
- Weighted F1-Score = 0.710
- Macro F1-Score = 0.670
- Confusion Matrix = 
[[169 101]
 [159 446]]
- False Positives (FP) = 101
- False Negatives (FN) = 159


# 2.4 Intent Prompting and Prediction

In [20]:
print("------Result of Person1 Intent------")
twointent_test = intent_p1(test)
print(twointent_test)

------Result of Person1 Intent------
                                              Dialogue  Manipulative  \
0    Person1: 657-2036\nPerson2: Hello -- who is th...             0   
1    Person1: Allison? Why can't remember what happ...             1   
2    Person1: Fox, Bud D. is this deja vu or has it...             1   
3    Person1: I have this thing where I have to kno...             1   
4    Person1: ...Come in here...I work here, I don'...             1   
..                                                 ...           ...   
870  Person1: Wait I...\nPerson2: You were paid to ...             1   
871  Person1: See! The big guy likes it.\nPerson2: ...             1   
872  Person1: All right, look.  I can make it Right...             1   
873  Person1: Uh, you first.\nPerson2: C'mon, Beavi...             0   
874  Person1: Hey!  I'm getting a little tired of t...             1   

     Prediction                                          Intent_p1  
0           0.0  Person1 info

In [21]:
print("------Result of Person2 Intent------")
twointent_test = intent_p2(twointent_test)
print(twointent_test)

------Result of Person2 Intent------
                                              Dialogue  Manipulative  \
0    Person1: 657-2036\nPerson2: Hello -- who is th...             0   
1    Person1: Allison? Why can't remember what happ...             1   
2    Person1: Fox, Bud D. is this deja vu or has it...             1   
3    Person1: I have this thing where I have to kno...             1   
4    Person1: ...Come in here...I work here, I don'...             1   
..                                                 ...           ...   
870  Person1: Wait I...\nPerson2: You were paid to ...             1   
871  Person1: See! The big guy likes it.\nPerson2: ...             1   
872  Person1: All right, look.  I can make it Right...             1   
873  Person1: Uh, you first.\nPerson2: C'mon, Beavi...             0   
874  Person1: Hey!  I'm getting a little tired of t...             1   

     Prediction                                          Intent_p1  \
0           0.0  Person1 inf

In [22]:
# two intent zeroshot预测
print("------Experiment: Classification Result of Two-intent Prompting------")
twointent_prediction(twointent_test)

------Experiment: Classification Result of Two-intent Prompting------
- Accuracy = 0.726
- Precision = 0.812
- Recall = 0.785
- Weighted F1-Score = 0.728
- Macro F1-Score = 0.685
- Confusion Matrix = 
[[160 110]
 [130 475]]
- False Positives (FP) = 110
- False Negatives (FN) = 130
